New method: employed in the SSD.py script

In [1]:
import neurokit2 as nk
import pandas as pd
import numpy as np
from preprocessing import *

In [5]:
PATH = "C:/Users/eleon/Desktop/Master/FJS 23/Master Project/sensor-data-analysis-pipeline/back-end/src/data/p1_wk1/1022102cFnorm.csv"
df = pd.read_csv(PATH)
print(f"dfshape: {df.shape}")
ecg = df["ECG"]

dfshape: (74155516, 9)


In [6]:
ecg

0           0.234100
1           0.234080
2           0.234100
3           0.234110
4           0.234090
              ...   
74155511   -0.001563
74155512   -0.001669
74155513   -0.001691
74155514   -0.001551
74155515   -0.001481
Name: ECG, Length: 74155516, dtype: float64

In [10]:
def peaks_to_rri(peaks, sampling_rate=2000, interpolate=False, **kwargs):
    rri = np.diff(peaks) / sampling_rate * 1000

    if interpolate is False:
        return rri

    else:

        # Minimum sampling rate for interpolation
        if sampling_rate < 10:
            sampling_rate = 10

        # Compute length of interpolated heart period signal at requested sampling rate.
        desired_length = int(np.rint(peaks[-1]))

        rri = nk.signal_interpolate(
            peaks[1:],  # Skip first peak since it has no corresponding element in heart_period
            rri,
            x_new=np.arange(desired_length),
            **kwargs
        )
        return rri, sampling_rate

In [11]:

def compute_HRV(rpeaks_clean, sampling_rate=2000, window=5):    
    # Regular HRV matrix (from peaks)
    duration_peaks=rpeaks_clean[len(rpeaks_clean)-1] #gives me the duration in samples
    divider=duration_peaks/sampling_rate/60/window  #sampling_rate, 5 min window segments
    segment=np.array_split(rpeaks_clean,divider) #divide in segments of 5 min; the last segment may be shorter; discard during statistical analysis on HRV metrics

    hrv_final_df=pd.DataFrame()

    for i in range(len(segment)):
        hrv_segment = nk.hrv(segment[i], sampling_rate=sampling_rate, show=False)
        print(i)
        hrv_final_df = pd.concat([hrv_final_df, hrv_segment],ignore_index=True)


    return hrv_final_df

In [12]:

# Filter the data with ranges specified from Barbara
filter_ecg = nk.signal_filter(ecg, sampling_rate=2000, lowcut=0.5, highcut=150)
# clean ecg signal
ecg_cleaned = nk.ecg_clean(filter_ecg, sampling_rate=2000, method="pantompkins1985")
# find r peaks
rpeaks, info = nk.ecg_peaks(ecg_cleaned, sampling_rate=2000)

In [13]:
# Trim trailing zeros
#rpeaks = np.trim_zeros(rpeaks,trim='b')
# Artifact removal
# nk.signal_fixpeaks saves the corrected peak locations to the [1] index of the output data sturcture
# accessible like so: clean_peaks[1]
rpeaks_clean=nk.signal_fixpeaks(rpeaks, sampling_rate=2000, iterative=False, show=False,interval_min=0.33,interval_max=0.75, method="kubios")
# Convert peaks to rri - not useful for our case
rri = peaks_to_rri(rpeaks_clean[1], sampling_rate=2000, interpolate=False)



In [14]:
rpeaks_clean

({'ectopic': [151,
   628,
   675,
   741,
   788,
   807,
   867,
   964,
   994,
   1107,
   1208,
   1212,
   1213,
   1227,
   1480,
   1851,
   1912,
   2061,
   2135,
   2238,
   2330,
   2388,
   2418,
   2482,
   2568,
   2900,
   3180,
   3203,
   3212,
   3855,
   3930,
   4152,
   4389,
   4460,
   4585,
   4767,
   4768,
   5089,
   5928,
   6401,
   6983,
   7054,
   7264,
   7326,
   7493,
   7573,
   7641,
   8137,
   8417,
   8555,
   8775,
   9000,
   9294,
   9543,
   9797,
   12346,
   12347,
   12465,
   12466,
   12555,
   12639,
   13752,
   14062,
   14376,
   14453,
   14525,
   15420,
   15421,
   16303,
   18895,
   19576,
   19911,
   20031,
   20300,
   20311,
   20314,
   20669,
   20824,
   20827,
   21068,
   21283,
   21404,
   22898,
   22904,
   24119,
   25447,
   25464,
   25944,
   26141,
   26237,
   26332,
   26346,
   26403,
   26501,
   26684,
   26940,
   27007,
   27234,
   27443,
   27908,
   28198,
   29252,
   29255,
   30332,
   30743,
   

In [15]:
rpeaks_clean = rpeaks_clean[1]

In [16]:
# Compute HRV metrics

duration_peaks=rpeaks_clean[len(rpeaks_clean)-1] #gives me the duration in samples
divider=duration_peaks/2000/60/5  #sampling_rate, 5 min window segments
segment=np.array_split(rpeaks_clean,divider) #divide in segments of 5 min; the last segment may be shorter; discard during statistical analysis on HRV metrics

hrv_segment_df=pd.DataFrame()

for i in range(len(segment)):
    hrv_segment = nk.hrv(segment[i], sampling_rate=2000, show=False)
    print(i)
    hrv_segment_df = pd.concat([hrv_segment_df, hrv_segment],ignore_index=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122


In [17]:
hrv_segment_df

,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,HRV_SDNNI5,HRV_RMSSD,HRV_SDSD,...,HRV_SampEn,HRV_ShanEn,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC
0,836.895349,76.570108,55.218731,55.739381,NaN,NaN,NaN,NaN,55.957737,56.049200,...,1.351648,7.784170,0.957438,1.482828,1.290018,1.884596,1.679477,1.779508,2.964053,0.711210
1,884.810631,76.444145,41.849409,63.504553,NaN,NaN,NaN,NaN,66.751473,66.863001,...,1.478874,7.720651,1.003162,1.520185,1.193988,1.941706,1.707561,1.827272,2.506061,0.711210
2,968.191030,47.784264,14.023097,45.597989,NaN,NaN,NaN,NaN,48.074751,48.152676,...,1.805182,7.574889,1.289141,1.719685,1.449445,2.138344,1.866833,1.857337,4.120724,0.930043
3,962.453488,46.320149,15.350349,43.167989,NaN,NaN,NaN,NaN,47.999167,48.075815,...,1.805904,7.401584,1.304632,1.589985,1.471922,1.963545,1.760790,1.873415,3.291300,0.902689
4,941.426910,58.114725,19.985262,55.601751,NaN,NaN,NaN,NaN,51.023303,51.107813,...,1.559517,7.439822,1.099741,1.555696,1.393714,1.919928,1.736137,1.737767,3.053254,0.984752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,1084.786667,141.104066,16.885084,71.823683,63.463900,121.706258,NaN,NaN,72.036217,72.153503,...,1.222944,7.782904,0.894902,0.811745,1.372599,1.464129,1.592989,1.746870,2.515396,0.932599
119,1106.238333,99.484141,17.029692,93.231215,17.696921,97.389570,NaN,NaN,81.103809,81.239774,...,1.901397,7.822022,1.210424,1.707332,1.353742,2.082453,1.869858,1.691459,3.544331,0.822882
120,1072.918333,150.723470,76.948120,130.577185,49.971752,132.513035,NaN,NaN,87.985880,88.133310,...,1.324994,7.824538,0.902823,1.287177,1.355464,1.803855,1.690993,1.610590,2.873527,0.685735
121,1069.996667,98.475936,15.394481,96.067567,11.393285,98.196971,NaN,NaN,68.681217,68.795500,...,1.633588,7.838754,1.114208,1.389331,1.329190,2.175825,1.739461,1.578537,3.198565,0.877741


In [18]:
def get_herzig_ranges():
    return {
        'nrem': {'min': 0.31, 'median': (2.02-0.31)/2 , 'max': 2.02},
        'rem' : {'min': 1.30, 'median': 2.02 , 'max': 3.22}
    }

## Sleep stage detection pipeline

In [19]:
window_size = 5*60*2000
start = 0
end = window_size
x=0
y=0
values = []
for i, row in hrv_segment_df.iterrows():
    print(row['HRV_LFHF'])
    nrem = get_herzig_ranges()['nrem']
    rem = get_herzig_ranges()['rem']


    if  nrem['min'] <= row['HRV_LFHF'] <= nrem['max']:
        stage = 'nrem'

    if row['HRV_LFHF'] < nrem['min']:
        stage = 'nrem'

    if rem['min'] <= row['HRV_LFHF'] <= rem['max']:
        stage = 'rem'

    if row['HRV_LFHF'] > rem['max']:
        stage = 'rem'

    values.append({
            'start_id': start,
            'end_id': end,
            'LF_HF': row['HRV_LFHF'],
            'SD': row['HRV_SDNN'],
            'stage': stage,
            'y': y,
            'x': x
        })

    if x==((90/5)-1):
        x=0
        y+=1

    else:
        x+=1

        

    start = end
    end += window_size


1.6426161983559298
1.073289488257166
1.7854041157820064
1.0764312804609886
3.2930966254137966
2.761378566375531
3.5291722328471513
1.9296602576098594
2.19897313617705
1.0050634302979329
2.7541428271284447
2.085263127326261
1.2908861715581825
2.3093883214491133
2.083360718283725
2.0541137799140152
2.0722785569658493
2.849536602343722
2.1240359094960195
1.2815198241622434
3.604467419087124
2.726092008890994
2.9129921351406125
3.8933583005814056
4.312430598894639
5.551411595800466
1.5951747615959244
0.8220086077040393
0.7670219210918903
1.6403496511103643
0.3231133095183337
0.9576152059854329
0.5628915470058222
1.296786260302575
1.59530007939732
0.81111757729386
0.4762534473845131
0.8665756606927525
2.3908075429487483
0.5845995063887968
0.22264885861987432
3.009292143632512
1.6404442514985877
1.289071226800965
4.1032833691250925
2.703553508143696
2.4788994767496013
1.6289788086026369
2.1802200661126645
0.41228228476232315
0.9564274319279885
3.008403682648634
0.5135649033938299
0.259742935

In [20]:
values

[{'start_id': 0,
  'end_id': 600000,
  'LF_HF': 1.6426161983559298,
  'SD': 76.57010847339781,
  'stage': 'rem',
  'y': 0,
  'x': 0},
 {'start_id': 600000,
  'end_id': 1200000,
  'LF_HF': 1.073289488257166,
  'SD': 76.44414531713261,
  'stage': 'nrem',
  'y': 0,
  'x': 1},
 {'start_id': 1200000,
  'end_id': 1800000,
  'LF_HF': 1.7854041157820064,
  'SD': 47.784263999100894,
  'stage': 'rem',
  'y': 0,
  'x': 2},
 {'start_id': 1800000,
  'end_id': 2400000,
  'LF_HF': 1.0764312804609886,
  'SD': 46.32014856183751,
  'stage': 'nrem',
  'y': 0,
  'x': 3},
 {'start_id': 2400000,
  'end_id': 3000000,
  'LF_HF': 3.2930966254137966,
  'SD': 58.11472538655982,
  'stage': 'rem',
  'y': 0,
  'x': 4},
 {'start_id': 3000000,
  'end_id': 3600000,
  'LF_HF': 2.761378566375531,
  'SD': 84.42618529887278,
  'stage': 'rem',
  'y': 0,
  'x': 5},
 {'start_id': 3600000,
  'end_id': 4200000,
  'LF_HF': 3.5291722328471513,
  'SD': 91.82600698738322,
  'stage': 'rem',
  'y': 0,
  'x': 6},
 {'start_id': 420000